In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [21]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from pymongo import MongoClient

from langchain_ollama import OllamaEmbeddings

from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

from langchain_community.tools.pubmed.tool import PubmedQueryRun

search = PubmedQueryRun()

llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')

In [ ]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

db_name = r"D:\NLP\LLM\Langchain and Ollama\09. Vector Stores and Retrievals\health_supplements"
vector_store = FAISS.load_local(db_name, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type = 'similarity', 
                                      search_kwargs = {'k': 3})

In [ ]:
### Tool Creation

from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults

@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information.
    Not for medical and life sciences queries
    
    Args:
    query: The search query
    """
    
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)
    return response

@tool
def health_supplements(query: str) -> str:
    """Search for information about Health Supplements.
    For any questions about Health and Gym Supplements, you must use this tool!,

    Args:
        query: The search query.
    """
    response = retriever.invoke(query)
    return response

@tool
def pubmed_search(query):
    """
    Search pubmed for medical and life sciences queries.
    
    Args:
    query: The search query
    """
    
    search = PubmedQueryRun()
    response = search.invoke(query)
    return response

@tool
def tavily_search(query):
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updates etc.
    
    Args:
    query: The search query
    """
    
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )
    response = search.invoke(query)
    return response

@tool
def get_assets_by_employee(employee: str) -> str:
    """
    Retrieve a list of assets assigned to a specific employee.

    Args:
        employee: The name of the employee to look up.
    """
    try:
        client = MongoClient("mongodb...url")
        db = client["rasa"]
        collection = db["employees"]

        employee_data = collection.find_one({
            "employee": {"$regex": f"^{employee}$", "$options": "i"}
        })

        if employee_data and "assets_assigned" in employee_data:
            assets = employee_data["assets_assigned"]
            if assets:
                return f"Assets assigned to {employee}: {', '.join(assets)}"
            else:
                return f"{employee} has no assets assigned."
        else:
            return f"No employee found with name '{employee}'."

    except Exception as e:
        return f"Database error: {str(e)}"


In [25]:
tools  = [pubmed_search,tavily_search,wikipedia_search,get_assets_by_employee]

llm_with_tools = llm.bind_tools(tools)
print(llm_with_tools)

bound=ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434') kwargs={'tools': [{'type': 'function', 'function': {'name': 'pubmed_search', 'description': 'Search pubmed for medical and life sciences queries.\n\nArgs:\nquery: The search query', 'parameters': {'properties': {'query': {}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'tavily_search', 'description': 'Search the web for realtime and latest information.\nfor examples, news, stock market, weather updates etc.\n\nArgs:\nquery: The search query', 'parameters': {'properties': {'query': {}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'wikipedia_search', 'description': 'Search wikipedia for general information.\nNot for medical and life sciences queries\n\nArgs:\nquery: The search query', 'parameters': {'properties': {'query': {}}, 'required': ['query'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'get_assets_by_employ

In [26]:
llm_with_tools.invoke("Tell me about esophageal cancer")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-06-23T18:24:04.9728496Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2136637000, 'load_duration': 112111800, 'prompt_eval_count': 370, 'prompt_eval_duration': 818000000, 'eval_count': 21, 'eval_duration': 1203000000, 'model_name': 'llama3.2:3b'}, id='run--85921614-aeac-45bf-89ed-85066a742157-0', tool_calls=[{'name': 'pubmed_search', 'args': {'query': 'esophageal cancer'}, 'id': '7302d35c-7a66-453c-9a1e-555ac8e4e94a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 370, 'output_tokens': 21, 'total_tokens': 391})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant which helps solve problems by calling tool functions."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

result = agent_executor.invoke({"input": "Tell me about x-ray machine"})
print(result)

{'input': 'Tell me about x-ray machine', 'output': "X-ray machines are devices that use X-rays for various applications, including medicine, industrial inspection, and material testing. In medical settings, X-ray machines are used to produce images of the internal structures of living organisms, helping doctors diagnose and treat conditions.\n\nHere's a brief overview:\n\n*   **How it works**: X-ray machines produce X-rays by accelerating electrons to high speeds and then decelerating them using magnetic fields.\n*   **Types of X-ray machines**: There are several types of X-ray machines, including digital X-ray systems, conventional X-ray systems, and computed tomography (CT) scanners.\n*   **Applications**: X-ray machines have a wide range of applications, including medical imaging, industrial inspection, security screening, and material testing.\n*   **Safety precautions**: X-ray machines can be hazardous to health if not used properly. It's essential to follow safety guidelines and 